In [1]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
Boston=pd.read_csv('housing.data',delim_whitespace=True, header=None)
Boston


<table style="width:60%">
<tr>
<th> Code :   </th>
<th> Description : </th>
</tr>

<tr>
<th> CRIM </th> 
<th> per capita crime rate per town  </th>
</tr>
<tr>
<th> ZN </th>
<th> proportion of residential land zones for lots over 25000 sgr ft ></th>
</tr>
<tr>
<th> INDUS </th>
<th> proportion of non retail business acr per town </th>
</tr>
<tr>
<th> CHAS</th>
<th> Charles River dummy variable (=1 if tract bounds river) </th>
</tr>
<tr>
<th> NOX </th>
<th> NO concentration (pp 10 million) </th>
</tr>
<tr>
<th> RM </th>
<th> Average N rooms  </th>
</tr>
<tr>
<th>  AGE  </th>
<th> proportion of owner occupied builts prior 1940 </th>
</tr>
<tr>
<th>  DIS  </th>
<th> weighted distances to 5 Boston employment centers </th>
</tr>

<tr>
<th>  RAD  </th>
<th> index of accessibility to radial highways </th>
</tr>
    
<tr>
<th>  TAX  </th>
<th> full value property tax rate per 10 000 $ </th>
</tr>
    
<tr>
<th>  PTRATIO </th>
<th> pupil teacher ratio by town </th>
</tr>
    
<tr>
<th> B </th>
<th> 1000(BK-0.63)² ; bk proportion of blacks by town </th>
</tr>
    
<tr>
<th> LSTAT </th>
<th> % lower status of the population </th>
</tr>
    
<tr>
<th>  MEDV  </th>
<th> median value of owner occupied homes in 1000 $ </th>
</tr>
</table>


In [ ]:
Boston.columns=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX',
                'PTRATIO','B','LSTAT','MEDV']
#Boston.to_csv('Boston.csv')
Boston.describe()


# Exploratory data Analysis (EDA) #

In [ ]:
sns.pairplot(Boston, height=1.5)
plt.show()


In [ ]:
Features=['CRIM','ZN','INDUS','NOX','RM']
sns.pairplot(Boston[Features], height=2.5)
plt.show()


In [ ]:
Features2=['AGE','TAX','PTRATIO','B','LSTAT','MEDV']
sns.pairplot(Boston[Features2], height=2.5)
plt.show()


# Correlation & Features selection #

In [ ]:
pd.options.display.float_format='{:,.3f}'.format
Boston.corr()


In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(Boston.corr(), annot=True)
plt.show()


In [ ]:

plt.figure(figsize=(12,10))
sns.heatmap(Boston[['CRIM','ZN','INDUS','CHAS','MEDV']].corr(), annot=True)
plt.show()


# Linear regression with Scikit-Learn #

## Five steps of in using Scikit-earn estimator API by Jacob T. Vanderplas: ##
<ol>
<li> Choose class model by importing the appropriate estimator class from Scikit-earn </li>
<li> Choose model hyperparameters by instanciating this class with desired values
<li> Arrange data into a features matrix and target vector
<li> Fit the model to ur data by calling the fit() method of the model instance
<li> Apply the model to the new data:
<ul>
<li> for supervised learning: often we predict labels for unknown data using predict() method
<li> for unsupervised learning: often we transform or infer properties of the data using the transform() or predict() method
</ul>
</ol>

### MEDV vs RM Linear regression ###

In [ ]:
from sklearn.linear_model import LinearRegression
X=Boston['RM'].values.reshape(-1,1)
Y=Boston['MEDV'].values


In [ ]:
model=LinearRegression()
model.fit(X,Y)
print(f'linear model coeficient={model.coef_} and B={model.intercept_}')


In [ ]:
plt.figure(figsize=(10,6))
sns.regplot(x=X,y=Y,color='orange')
plt.xlabel('Average N of rooms')
plt.ylabel('owner occupied houses mediane value 1000$')
plt.show()


In [ ]:
sns.jointplot(x='RM', y='MEDV', data=Boston, kind='reg',height=10,color='orange')
plt.grid()
plt.show()


In [ ]:

model.predict(np.array([5]).reshape(1,-1))


### LSTAT vs MEDV Linear regression ###

In [ ]:

X2=Boston['LSTAT'].values.reshape(-1,1)
Y2=Boston['MEDV'].values
model2=LinearRegression()
model2.fit(X2,Y2)
print(f'linear, a={model2.coef_} and b={model2.intercept_}')


In [ ]:
plt.figure(figsize=(10,6))
sns.regplot(x=X2,y=Y2,color='DeepSkyBlue');
plt.xlabel('Median value of owner occupied homes in 1000 $')
plt.ylabel('% lower status of the population')
plt.grid()
plt.show();


In [ ]:
#optional
sns.jointplot(x='LSTAT', y='MEDV', data=Boston, kind='reg',height=10,color='lime')
plt.grid()
plt.show()


In [ ]:
model2.predict(np.array([15]).reshape(1,-1))


# Robust Regression #

## Random Sample Consensus (RANSAC) ##
<ol>
<li> select <strong> min_samples </strong>random samples from original data and check whether the dataset is valid <strong>(is_data_valid)</strong> </li>
<li> fit a model to a random subset <strong>(base_estimator.fit)</strong> and check whether the estimated model is valid <strong> (is_model_valid)</strong> </li>
<li> Classify all data as inliers or ouliers by calculating the residuals to the estimated model <strong>(base_estimator.predict(x)-y)</strong> all data samples
with absolute residuals smaller than the <strong>residual_threshold </strong>are considered as inliers </li>
<li> save fitted model as best model if number of outliers samples is maximal, in case the current estimated model has the same number of inliers,
    it is considered as the best model if it has better score</li>
</ol>

In [ ]:
from sklearn import linear_model
from sklearn import datasets
#RM vs MEDV
#X=Boston['RM'].values.reshape(-1,1)
#Y=Boston['MEDV'].values
coef=True
# Robustly fit linear model with RANSAC algorithm
ransac = linear_model.RANSACRegressor()
ransac.fit(X,Y)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Compare estimated coefficients
print(f'Estimated coefficients (coef:{coef} , linear regression: {model.coef_},RANSAC= {ransac.estimator_.coef_}x+{ransac.estimator_.intercept_})')


In [ ]:
#predictions and plotting
line_x=np.arange(3,10,1)
line_y_ransac=ransac.predict(line_x.reshape(-1,1))
line_y=model.predict(line_x.reshape(-1,1))

sns.set(style='darkgrid', context='notebook')
plt.figure(figsize=(12,8))
plt.scatter(X[inlier_mask], Y[inlier_mask], c='blue', marker='o', label='Inliers')
plt.scatter(X[outlier_mask], Y[outlier_mask], c='violet', marker='s', label='Outliers')
plt.plot(line_x, line_y, color='grey',linewidth=2, label="Linear regressor")
plt.plot(line_x,line_y_ransac,color="red", linewidth=2,label="RANSAC regressor")
plt.xlabel('Average N rooms')
plt.ylabel('median value of owner occupied homes in 1000 $ ')
plt.legend(loc='lower right')
plt.show()


**linear regression Assumptions:** https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/assumptions-of-linear-regression/

# Evaluate Regression Model Performance #

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X=Boston.iloc[:, :-1].values
Y=Boston['MEDV'].values

x_train, x_test,y_train, y_test=train_test_split(X, Y, test_size=0.2, random_state=0)
lr=LinearRegression()
lr.fit(x_train,y_train)
y_train_pred=lr.predict(x_train)
y_test_pred=lr.predict(x_test)


### Method1: Residual Analysis ###

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(y_train_pred, y_train_pred-y_train, c='blue', marker='s', label='Training')
plt.scatter(y_test_pred,y_test_pred-y_test, c='orange', marker='o', label='Test')
plt.xlabel('Predicted values')
plt.ylabel('Residuals ')
plt.legend(loc='upper right')
plt.hlines(y=0, xmin=-10, xmax=50, lw=2, color='k')
plt.xlim([-10,50])
plt.show()


### Method2: Mean Squarred Error (MSE) ###
<ul>
<li>The average value of the sums of the squarred error cost function </li>
<li>Useful for comparing different regression models </li>
<li>For tuning parameters via a grid search and cross-validation </li>
</ul>

## $$MSE={{1}\over{n }}{\sum_{i=1}^n{(yi-ŷi)²}}$$ ##

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_train,y_train_pred)
mean_squared_error(y_test,y_test_pred)


### Method3: Coefficient of determination R² ###
### $$ R²=1-{SSE \over SST}$$ ###
<ul>
<li>SSE: Sum of squarred errors </li>
<li>SST: Total sum of squares </li>

</ul>


In [ ]:
from sklearn.metrics import r2_score

r2_score(y_train,y_train_pred)
r2_score(y_test,y_test_pred)


# The perfect model #

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

generate_random=np.random.RandomState(0)
x=10*generate_random.rand(1000)
y=3*x+np.random.randn(1000)
plt.figure(figsize=(10,8))
plt.scatter(x,y, marker='^', c='indigo')
plt.show()


In [ ]:
x_train, x_test,y_train, y_test=train_test_split(x,y, test_size=0.3, random_state=0)
model=LinearRegression()
model.fit(x_train.reshape(-1,1),y_train)
y_train_pred=model.predict(x_train.reshape(-1,1))
y_test_pred=model.predict(x_test.reshape(-1,1))


In [ ]:
#Residual Analysis

plt.figure(figsize=(12,8))
plt.scatter(y_train_pred, y_train_pred-y_train, c='blue', marker='s', label='Training')
plt.scatter(y_test_pred,y_test_pred-y_test, c='orange', marker='o', label='Test')
plt.xlabel('Predicted values')
plt.ylabel('Residuals ')
plt.legend(loc='upper right')
plt.hlines(y=0, xmin=3, xmax=33, lw=2, color='r')
plt.xlim([-5,35])
plt.ylim([-25,15])
plt.show()


In [ ]:
#MSE

from sklearn.metrics import mean_squared_error

mean_squared_error(y_train,y_train_pred)
mean_squared_error(y_test,y_test_pred)
print(f'MSE train:{mean_squared_error(y_train,y_train_pred)} , MSE Test:{mean_squared_error(y_test,y_test_pred)}')


In [ ]:
#R²
from sklearn.metrics import r2_score

r2_score(y_train,y_train_pred)
r2_score(y_test,y_test_pred)
print(f'R² train:{mean_squared_error(y_train,y_train_pred)} , R² Test:{mean_squared_error(y_test,y_test_pred)}')


# Multiple Regression #

## y=$\beta$0+$\beta1$*x1+$\beta2$*x2... ##
 

In [ ]:

Housing=pd.read_csv('housing.data',delim_whitespace=True, header=None)
Housing.columns=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX',
                'PTRATIO','B','LSTAT','MEDV']
Boston=Housing.iloc[:,0:13]

X=Boston
Y=Housing.iloc[:,13:15]
print(f'x= \n {X} and y=\n {Y}')


# Statsmodels #

In [ ]:
import statsmodels.api as sm

X_constant=sm.add_constant(X)
pd.DataFrame(X_constant)
#sm.OLS?
model=sm.OLS(Y,X_constant)
lr=model.fit()
lr.summary()
#P must be <0.025


## Summary: ##
<ul>
<li> <b>coefficient:</b> it is the value of the intercept. For each variable, it is the measurement of how change in that variable affects the independent variable. It is the ‘m’ in ‘y = mx + b’ One unit of change in the dependent variable will affect the variable’s coefficient’s worth of change in the independent variable. </li>
<li><b>std error:</b> is an estimate of the standard deviation of the coefficient, a measurement of the amount of variation in the coefficient throughout its data points. A low std error compared to a high coefficient produces a high t statistic, which signifies a high significance for your coefficient.</li>
<li><b>P>|t|</b> is one of the most important statistics in the summary. It uses the t statistic to produce the p-value, a measurement of how likely your coefficient is measured through our model by chance. The p-value of 0.378 for Wealth is saying there is a 37.8% chance the Wealth variable has no affect on the dependent variable, Lottery, and our results are produced by chance. A common alpha is 0.05, which few of our variables pass in this instance.</li>
<li><b>[0.025 and 0.975]</b> are both measurements of values of our coefficients within 95% of our data, or within two standard deviations. Outside of these values can generally be considered outliers.</li> </ul>
The p-value is the smallest test size that would cause an observation of t=0.1 to lead to a rejection of the null hypothesis

### Residual Tests: ###
<ul>
<li><b>Omnibus:</b> a combined statistic test for skewness and kurtosis</li>
<li><b>prob(Omnibus):</b> P-value of Omnibus test</li>
<li><b>Skewness:</b> a measure of symmetry of residuals around the mean.Zero if symmetrical. A positive value indicates a long tail to the right, a negative value indicates a long tail to the left</li>
<li><b>Kurtosis:</b> A measure of the shape of distribution of the residuals, A normal distribution has 0 measure.A negative value points to a flatter than normal distribution, a positive one has a higher peak than normal distribution</li>
<li><b>Durbin-Watson:</b> A test for presence of correlation among the residuals, this is important for time series modelling</li>
<li><b>Jarque-Bera:</b> it is a combined statistical test of Skewness and Kurtosis</li>
<li><b>Prob(JB): </b>p_value of Jarque-Bera</li>
<li><b>Cond.No: </b>it is a test for multicolinearity. > 30 indicates unstable results.</li>
    
</ul>


In [ ]:
import statsmodels.formula.api as smf

form_lr=smf.ols(formula='Y ~ CRIM+ZN+INDUS+CHAS+NOX+RM+AGE+DIS+RAD+TAX+PTRATIO+B+LSTAT',data=Housing)
mlr=form_lr.fit()
mlr.summary()


In [ ]:
#predicting 100 rows in basis of CRIM and Black
model_ex=smf.ols(formula='Y ~ CRIM+B',data=Housing) #CRIM+ZN+CHAS+
mlr_ex=model_ex.fit()
mlr_ex.summary()
#predictions = mlr_ex.predict(Housing[0:100])
#predictions.describe()


# Correlation Matrix #

In [2]:

Housing=pd.read_csv('housing.data',delim_whitespace=True, header=None)
Housing.columns=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX',
                'PTRATIO','B','LSTAT','MEDV']
Boston=Housing.iloc[:,0:13]
X=Boston
Y=Housing.iloc[:,13:15]


In [ ]:
pd.options.display.float_format='{:,.2f}'.format
corr_matrix=Boston.corr()
corr_matrix


In [ ]:
corr_matrix[np.abs(corr_matrix)< 0.6]=0 #lesser than 0.6 and greater than -0.6
corr_matrix

In [ ]:
palette = sns.color_palette('tab20b',10) # Default color palette
sns.palplot(palette) # Plotting your palette!
#sns.palplot(sns.color_palette('husl', 20)) # Seaborn color palette, with number of colors 
#sns.color_palette('rocket', as_cmap=True) # Get a CMap

plt.figure(figsize=(18,9))
sns.heatmap(corr_matrix, annot=True, cmap=palette)
plt.show()


# Detecting colinearity with Eigenvectors #

In [68]:
eigenvalues , eigenvectors= np.linalg.eig(Boston.corr())
pd.Series(eigenvalues).sort_values()
np.abs(pd.Series(eigenvectors[:,8])).sort_values(ascending=False)
print(Boston.columns[2],Boston.columns[8],Boston.columns[9])


INDUS RAD TAX


small values= presence of colinearity

# Revising Feature importance and Extractions #

In [ ]:
plt.hist(Boston['TAX'])
#plt.hist(Boston['NOX'])


# Standardise variable to identify Key Features #

In [116]:
from sklearn.linear_model import LinearRegression
pd.options.display.float_format='{:,.4f}'.format

model=LinearRegression()
model.fit(X,Y)
mc=list(np.transpose(model.coef_))
bc=list((Boston.columns))
bcc=np.transpose(bc)
result=pd.DataFrame({'Name':bc,'Coefficient':mc}).set_index('Name')
co=np.abs(result).sort_values(by=['Coefficient'], ascending=False)
co


,Coefficient
Name,
NOX,[17.766611228299993]
RM,[3.809865206809209]
CHAS,[2.686733819344964]
DIS,[1.4755668456002538]
PTRATIO,[0.9527472317072886]
LSTAT,[0.524758377855484]
RAD,[0.30604947898517126]
CRIM,[0.10801135783679763]
ZN,[0.04642045836688188]


### 2nd method ###

In [111]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
scaler=StandardScaler()
standard_coefficient_linear_reg=make_pipeline(scaler,model)
standard_coefficient_linear_reg.fit(X,Y)
scl=list(np.transpose(standard_coefficient_linear_reg.steps[1][1].coef_))
result=pd.DataFrame({'Name':bcc,'Coefficient':scl}).set_index('Name')
co=np.abs(result).sort_values(by=['Coefficient'], ascending=False)
co


,Coefficient
Name,
LSTAT,[3.7436271264671093]
DIS,[3.10404425808644]
RM,[2.6742301652393135]
RAD,[2.6622176424736246]
TAX,[2.0767816838433744]
PTRATIO,[2.060606658906759]
NOX,[2.056718266005216]
ZN,[1.081568627822379]
CRIM,[0.9281460643011983]


## Use R² to identify key features ##
<ul>
<li>Compare R² of the model with R² without a feature</li>
<li>significant change in R² means the importance of the feature</li>
</ul>

In [118]:
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf

linear_reg=smf.ols(formula='Y ~ CRIM+ZN+INDUS+CHAS+NOX+RM+AGE+DIS+RAD+TAX+PTRATIO+B+LSTAT',data=Boston)
Benchmark=linear_reg.fit()
r2_score(Y,Benchmark.predict(Boston))


0.7406426641094094

## Without LSTAT ##

In [120]:
linear_reg=smf.ols(formula='Y ~ CRIM+ZN+INDUS+CHAS+NOX+RM+AGE+DIS+RAD+TAX+PTRATIO+B',data=Boston)
lr_lstat=linear_reg.fit()
r2_score(Y,lr_lstat.predict(Boston))


0.6842042799773889

## Without AGE ##

In [122]:
linear_reg=smf.ols(formula='Y ~ CRIM+ZN+INDUS+CHAS+NOX+RM+DIS+RAD+TAX+PTRATIO+B+LSTAT',data=Boston)
lr_AGE=linear_reg.fit()
r2_score(Y,lr_AGE.predict(Boston))


0.7406412165505145